In [1]:
import torch
import numpy as np
import pandas as pd
from transformers import AutoModel, AutoTokenizer, DataCollatorWithPadding
from datasets import load_dataset
from sklearn.model_selection import train_test_split, StratifiedKFold
from datasets import load_dataset
from transformers import DataCollatorWithPadding
import random
from tqdm import tqdm

2022-08-18 12:09:53.871778: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
MAX_LEN = 512
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name_list = [
    "kbalbert_agument_epoch1_fold5_without_papago.pt",
    "kbalbert_agument_epoch3_fold5_without_papago.pt",
    "kbalbert_agument_epoch5_fold5_without_papago.pt",
    "kbalbert_origin_epoch1_fold5.pt",
    "kbalbert_origin_epoch3_fold5.pt"
]

In [3]:
kb_albert_model_path = "./models/kb-albert-char-base-v2"
albert = AutoModel.from_pretrained(kb_albert_model_path)
tokenizer = AutoTokenizer.from_pretrained(kb_albert_model_path)

tokenizer.truncation_side = "left"

Some weights of the model checkpoint at ./models/kb-albert-char-base-v2 were not used when initializing AlbertModel: ['predictions.LayerNorm.bias', 'predictions.bias', 'predictions.LayerNorm.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'sop_classifier.classifier.bias', 'predictions.dense.weight', 'sop_classifier.classifier.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
class ClassificationHead(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.dropout = torch.nn.Dropout(0.25)
        self.out_proj = torch.nn.Linear(768, 2)
    
    def forward(self, features):
        # 보통 분류기에선 start 토큰에 분류 결과를 담음
        x = features[:, 0, :]    # take <s> token (equiv. to [CLS])
        x = x.reshape(-1, x.size(-1))
        x = self.dropout(x)

        x = self.out_proj(x)
        return x

class AInalyst(torch.nn.Module):
    def __init__(self, pretrained_model):
        super(AInalyst, self).__init__()
        self.pretrained = pretrained_model
        self.classifier = ClassificationHead()
    
    def forward(self, input_ids=None, attention_mask=None, labels=None):
        outputs = self.pretrained(
            input_ids=input_ids,
            attention_mask=attention_mask,
            # labels=labels
        )
        self.labels = labels
        logits = self.classifier(outputs["last_hidden_state"])
        # prob = torch.nn.functional.softmax(logits, dim=-1)
        
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)
            return logits, loss
        else:
            return logits

In [5]:
import copy

load_model_list = list()

for model_name in model_name_list:
    load_model = copy.deepcopy(AInalyst(pretrained_model=albert))
    load_model = torch.nn.DataParallel(load_model)
    load_model.to(device)
    
    now_state = torch.load(f"./models/{model_name}")    
    load_model.load_state_dict(now_state)
    load_model.eval()
    
    load_model_list.append(load_model)

In [6]:
def inference_tokenized_fn(data):
    outputs = tokenizer(data["article"], padding="max_length", max_length=MAX_LEN, truncation=True)
    return outputs

inference_dataset = load_dataset("csv", data_files=f"./data/inference_data.csv")["train"]
inference_dataset = inference_dataset.map(inference_tokenized_fn,
                                          remove_columns=["Unnamed: 0", "company", "title", "opinion", "firm", "date", "article"])

Using custom data configuration default-68bfb0a5382dece9
Reusing dataset csv (/home/piai/.cache/huggingface/datasets/csv/default-68bfb0a5382dece9/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/piai/.cache/huggingface/datasets/csv/default-68bfb0a5382dece9/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-e1e08dbd646bbdf1.arrow


In [7]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

inference_dataloader = torch.utils.data.DataLoader(
    inference_dataset,
    sampler = torch.utils.data.SequentialSampler(inference_dataset),
    batch_size = 1,
    collate_fn = data_collator,
)

In [8]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [9]:
probabilities = [list() for _ in model_name_list]
predictions = [list() for _ in model_name_list]

for step, batch in enumerate(tqdm(inference_dataloader, desc="inference", mininterval=0.1)):    
    batch_input_ids = batch["input_ids"].to(device)
    batch_attention_mask = batch["attention_mask"].to(device)

    for idx, load_model in enumerate(load_model_list):
        with torch.no_grad():
            logits = load_model(
                input_ids = batch_input_ids,
                attention_mask = batch_attention_mask,
            )

            prob = torch.nn.functional.softmax(logits, dim=-1)
            predict = torch.argmax(prob, axis=1)
            
            if predict == 1:
                prob = np.round(np.max(prob.detach().cpu().numpy(), axis=1) * 100, 2)
            else:
                prob = np.round((1 - np.max(prob.detach().cpu().numpy(), axis=1)) * 100, 2)
                
            predict = predict.detach().cpu().numpy()

            probabilities[idx].append(prob[0])
            predictions[idx].append(predict[0])

inference: 100%|████████████████████████| 50083/50083 [1:36:42<00:00,  8.63it/s]


In [10]:
ensemble_prob = np.mean(probabilities, axis=0, dtype=np.float64)
ensemble_pred = (ensemble_prob > 50).astype(np.int32)

inference_dataframe = pd.read_csv("./data/inference_data.csv")

convert_pred = list(map(lambda x: "매수" if x == 1 else "매도", ensemble_pred))
convert_prob = list(map(lambda x: np.round(x, 2) if x > 50 else np.round(100 - x, 2), ensemble_prob))

inference_dataframe = inference_dataframe.drop(labels="Unnamed: 0", axis=1)
inference_dataframe["predictions"] = convert_pred
inference_dataframe["pred_rate"] = convert_prob
inference_dataframe.to_csv(f"./data/ensemble_inference_data.csv", index=False)

## Test output

In [11]:
for idx, (_, element) in enumerate(inference_dataframe.iterrows()):
    if idx == 30:
        break
    print(element["predictions"], element["pred_rate"])
    print(element["article"])
    print("=" * 100)

매수 99.93


투자의견 BUY, 목표주가 480,000원 유지!동사의 투자의견 매수와 목표주가 를 유지한다. 상반기 스마트폰 시장 침체 에도 동사의 실적은 견조했다. 1) 하반기 북미 고객사의 신모델 출시가 기대되는 상황이며 , 2) 동사는 차질없이 물량을 공급하고 있는 것으로 파악된다. 3) 즉 , 하반기 실적 가시성은 확보 되었다. 전방 시장의 불확실성 에도 동사는 안전한 투자처가 될 것으로 판단된다 . 22년 역대 최대 실적을 기록할 전망이며 현재 12MF EPS 기준 PER 7.4배다. 여전히 주가 매력도 가 높고 실적 대비 저평가 구간이다 . 따라서 조정은 위기가 아닌 매수기회다.

매수 78.83


투자의견 BUY, 목표주가 220,000원 유지동사의 투자의견 매수와 목표주가 를 유지 한다. 1) 패키지 기판 수요 는 여전히 타이트한 상황이며 3 분기에도 현 기조 를 유지할 전망이다. 2) MLCC 재고조정은 3 분기까지 지속되나 이는 MLCC 재고수준을 정상화하는 기간으로 판단된다. 2 분기는 MLCC 사이클 저점에서 동사의 기초체력을 재평가는 시기였다. 불확실한 수요가 지속되는 가운데 실적 비수기인 상반기가 지났다 . 하반기 는 IT 성수기 이므로 기저효과 가 기대된다.

매도 75.88


전망치 상향 조정으로 목표주가 53만원으로 상향LG에너지솔루션에 대한 투자의견 BUY 를 유지하고 목표주가는 기존 50 만원에서 53만원으로 상향한다. 적용된 멀티플은 18 배로 하향하나 판가 가격전가력 확대 /CAPA 상향 등으로 2024 년 EBITDA 추정치를 상향한 결과다. 2 분기 수익성은 중국 코로나 락다운 여파와 원가 판가 전가시점의 차이로 하락했으나 하반기 고객사향 출하 회복 , GM 향 물량 추가 , 스프레드 개선 등으로 매출액 수익성이 동반 개선될 전망이다.

매수 57.68


2Q22 Review : 높아진 컨센서스 상회LG이노텍의 2분기 매출액 3조 7,026억원(YoY+57.2%, QoQ-31.0%), 영업이익 2,899억원(